In [1]:
#%pip install pyspark
#%pip install delta-spark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import DataFrame
from pyspark.sql.streaming import DataStreamWriter
from minio import Minio
from delta.tables import *
import os

In [3]:
spark = (
        SparkSession.builder
            .master("spark://spark-master-svc.lakehouse.svc.cluster.local:7077")
            .appName("raw_to_bronze")
            ## Config Fields
            .config('spark.sql.debug.maxToStringFields', 5000)
            .config('spark.debug.maxToStringFields', 5000)
            ## Optimize
            .config("delta.autoOptimize.optimizeWrite", "true")
            .config("delta.autoOptimize.autoCompact", "true")
            ## Delta Table
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            # Resources
            .config("spark.executor.memory", "2g")
            .config("spark.dynamicAllocation.enabled", "true")
            #.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            ## MinIO
            #.config("spark.hadoop.fs.s3a.endpoint", "http://tenant-lakehouse-hl.lakehouse.svc.cluster.local:9000")
            #.config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
            .config("spark.hadoop.fs.s3a.proxy.host", "tenant-lakehouse-hl.lakehouse.svc.cluster.local")
            .config("spark.hadoop.fs.s3a.proxy.port", "9000")
            .config("spark.hadoop.fs.s3a.access.key", "mIQD7asW927IUQst")
            .config("spark.hadoop.fs.s3a.secret.key", "yBCgkZNdoG49SL8iowq4Uv2AmtDHd6G0")
            .config("spark.hadoop.fs.s3a.path.style.access", "true")
            #.config("spark.hadoop.fs.s3a.connection.ssl.enabled", "true")
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            #.config("spark.hadoop.fs.s3a.attempts.maximum", "1")
            .config("spark.hadoop.fs.s3a.connection.estabilish.timeout", "5000")
            #.config("spark.hadoop.fs.s3a.connection.timeout", "10000")
            ## Jars
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-core_2.12:2.0.0")
            #.config("spark.jars", "/Users/thiagorente/Documentos/Python/lakehouse/spark/jars/hadoop-common-3.3.4.jar, \
            #                       /Users/thiagorente/Documentos/Python/lakehouse/spark/jars/hadoop-mapreduce-client-core-3.3.4.jar, \
            #                       /Users/thiagorente/Documentos/Python/lakehouse/spark/jars/hadoop-client-3.3.4.jar, \
            #                       /Users/thiagorente/Documentos/Python/lakehouse/spark/jars/aws-java-sdk-bundle-1.12.262.jar, \
            #                       /Users/thiagorente/Documentos/Python/lakehouse/spark/jars/hadoop-aws-3.3.4.jar")
            #.config("spark.driver.extraClassPath", \
            #                      "/Users/thomaz_rossito/driver_connection/jars/mysql-connector-j-8.0.32.jar")
            ## Hive SQL
            .enableHiveSupport()
            .getOrCreate()
    )

In [4]:
spark

In [5]:
log4jLogger = spark.sparkContext._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)
spark.sparkContext.setLogLevel("ERROR")

In [6]:
spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)

In [7]:
source_bucket = "raw"
prefix_bucket = "opencritic"
source_path = f"s3a://{source_bucket}/{prefix_bucket}/2024-08-02_20:19:03.332882/data_opencritic.json"


In [8]:
#games = spark.read.format('json').load(source_path)

games = spark.read.json('/tmp/data.json')
games

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/jovyan/.local/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/.local/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df_raw = spark \
        .readStream \
        .format("json") \
        .option("checkpointLocation", f"{source_path}/_checkpoint") \
        .load(source_path)

In [ ]:
spark.stop()